## HLA-A 

In [ ]:
from Bio import AlignIO
import numpy as np
import math
from collections import Counter

# Function to calculate Shannon entropy for a given column in the alignment
def shannon_entropy(column):
    counts = Counter(column)
    probabilities = [freq / len(column) for freq in counts.values()]
    entropy = -sum(p * math.log2(p) if p > 0 else 0 for p in probabilities)
    return entropy

# Function to normalize entropy based on the number of residues (with and without gaps)
def normalize_entropy(entropies, num_residues):
    max_entropy = math.log2(num_residues)
    normalized_entropies = [e / max_entropy for e in entropies]
    return normalized_entropies

# Load the aligned sequences using Biopython
aligned_sequences = AlignIO.read('aligned_A.fasta', 'fasta')

# Convert aligned sequences to a list of strings for easier processing
aligned_seq_str = [str(record.seq) for record in aligned_sequences]

# Transpose the alignment for column-wise processing
transposed_alignment = np.array([list(seq) for seq in aligned_seq_str]).T

# Calculate entropy for each column
entropies = [shannon_entropy(column) for column in transposed_alignment]

# Find the number of unique residues (with and without gaps)
unique_residues_with_gaps = len(set(''.join([''.join(column) for column in transposed_alignment])))
unique_residues_without_gaps = len(set(''.join([''.join(column) for column in transposed_alignment])) - set('-'))

# Normalized entropy values
normalized_entropies_with_gaps = normalize_entropy(entropies, unique_residues_with_gaps)
normalized_entropies_without_gaps = normalize_entropy(entropies, unique_residues_without_gaps)

# Print entropy values for each position
for i, (entropy, norm_with_gaps, norm_without_gaps) in enumerate(zip(entropies, normalized_entropies_with_gaps, normalized_entropies_without_gaps)):
    print(f"Position {i+1}: Entropy = {entropy:.4f}, Normalized (with gaps) = {norm_with_gaps:.4f}, Normalized (without gaps) = {norm_without_gaps:.4f}")

# Overall normalized entropy
overall_entropy_with_gaps = np.mean(normalized_entropies_with_gaps)
overall_entropy_without_gaps = np.mean(normalized_entropies_without_gaps)

print(f"\nOverall Normalized Entropy (with gaps): {overall_entropy_with_gaps:.4f}")
print(f"Overall Normalized Entropy (without gaps): {overall_entropy_without_gaps:.4f}")

In [ ]:
from Bio import AlignIO
from collections import Counter

# Define ANSI escape sequences for colors
RED = '\033[91m'
ENDC = '\033[0m'

# Load the alignment file
alignment = AlignIO.read("aligned_A.fasta", "fasta")

# Generate consensus sequence list
consensus_seq = []
for i in range(len(alignment[0])):
    column = [record.seq[i] for record in alignment]  # Extract the column
    most_common = Counter(column).most_common(1)[0][0]
    consensus_seq.append(most_common)

# Initialize the colored consensus sequence as an empty string
colored_consensus = ""

# Iterate over the consensus sequence and add color
for position, amino_acid in enumerate(consensus_seq, start=1):
    # Highlight every 10th amino acid
    if position % 10 == 0:
        colored_consensus += RED + amino_acid + ENDC
    else:
        colored_consensus += amino_acid

    # Every 100th character (including gaps), insert a newline
    if position % 50 == 0:
        colored_consensus += '\n'

# Print the colored consensus sequence
print(colored_consensus)

In [ ]:
from Bio import AlignIO
import numpy as np
import math
from collections import Counter

# Function to calculate probability distribution for a given column
def probability_distribution(column):
    counts = Counter(column)
    total = sum(counts.values())
    return {residue: count / total for residue, count in counts.items()}

# Function to calculate cross-entropy between two columns
def cross_entropy(dist1, dist2):
    residues = set(dist1.keys()).union(dist2.keys())
    return -sum(dist1.get(residue, 0) * math.log2(dist2.get(residue, 0) if dist2.get(residue, 0) > 0 else 1) for residue in residues)

# Load the aligned sequences
try:
    aligned_sequences = AlignIO.read('aligned_A.fasta', 'fasta')
except Exception as e:
    print(f"Error loading alignment file: {e}")
    raise

# Convert aligned sequences to a list of strings
aligned_seq_str = [str(record.seq) for record in aligned_sequences]

# Check if the alignment is correctly loaded
if not aligned_seq_str:
    print("No sequences found in the alignment.")
else:
    print(f"Loaded {len(aligned_seq_str)} sequences.")

# Transpose the alignment for column-wise processing
transposed_alignment = np.array([list(seq) for seq in aligned_seq_str]).T

# Calculate probability distributions for each column
prob_distributions = [probability_distribution(column) for column in transposed_alignment]

# Calculate cross-entropy for each pair of columns
cross_entropy_values = np.zeros((len(prob_distributions), len(prob_distributions)))
for i, dist1 in enumerate(prob_distributions):
    for j, dist2 in enumerate(prob_distributions):
        cross_entropy_values[i, j] = cross_entropy(dist1, dist2)

# Normalize cross-entropy values
max_entropy = math.log2(len(prob_distributions))
normalized_cross_entropy = cross_entropy_values / max_entropy

# Print cross-entropy values
print("Cross-Entropy Values:")
for i in range(len(cross_entropy_values)):
    for j in range(len(cross_entropy_values)):
        print(f"Cross-Entropy between positions {i+1} and {j+1}: {cross_entropy_values[i, j]:.4f}, Normalized: {normalized_cross_entropy[i, j]:.4f}")

# Create a heatmap using Plotly
fig = go.Figure(data=go.Heatmap(
    z=cross_entropy_values,
    x=[f"Pos {i+1}" for i in range(len(cross_entropy_values))],
    y=[f"Pos {j+1}" for j in range(len(cross_entropy_values))],
    colorscale='Viridis'))

fig.update_layout(
    title='Cross-Entropy Heatmap between Positions in Protein Alignment',
    xaxis_nticks=36,
    yaxis_nticks=36,    
    width=1000,  
    height=1000,  
    margin=dict(l=100, r=100, t=100, b=100)  
)

fig.show()

In [ ]:
import numpy as np
from Bio import AlignIO
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
from multiprocessing import Pool, cpu_count

def encode_sequences(sequences, alphabet='ACDEFGHIKLMNPQRSTVWY-'):
    char_to_int = {c: i for i, c in enumerate(alphabet)}
    encoded_seqs = [[char_to_int.get(char, -1) for char in seq] for seq in sequences]
    return np.array(encoded_seqs, dtype=int)

def calculate_mi_for_position(encoded_seqs, i):
    X = np.delete(encoded_seqs, i, axis=1)
    y = encoded_seqs[:, i]
    mi_score = mutual_info_classif(X, y, discrete_features=True)
    return np.insert(mi_score, i, 0)  # Insert 0 at the ith position for the diagonal

def main():
    aligned_sequences = AlignIO.read("aligned_A.fasta", "fasta")
    sequences = [str(record.seq) for record in aligned_sequences]
    encoded_seqs = encode_sequences(sequences)

    num_features = encoded_seqs.shape[1]
    mi_matrix = np.zeros((num_features, num_features))

    with Pool(processes=cpu_count()) as pool:
        results = pool.starmap(calculate_mi_for_position, [(encoded_seqs, i) for i in range(num_features)])

    for i, mi_scores in enumerate(results):
        mi_matrix[i, :] = mi_scores

    # Normalize the mutual information matrix using MinMaxScaler
    scaler = MinMaxScaler()
    mi_matrix_normalized = scaler.fit_transform(mi_matrix)

    # Create a heatmap using Plotly
    fig = go.Figure(data=go.Heatmap(
        z=mi_matrix_normalized,
        x=[f"Pos {i+1}" for i in range(num_features)],
        y=[f"Pos {i+1}" for i in range(num_features)],
        colorscale='Portland',  # Changed to Viridis for better color contrast
        zmin=0, zmax=1  # Set the scale from 0 to 1
    ))

    fig.update_layout(
        title='MinMax Scaled Mutual Information Heatmap',
        xaxis_title="Position",
        yaxis_title="Position",
        width=900,
        height=900
    )

    fig.show()

if __name__ == '__main__':
    main()

In [ ]:
from Bio import AlignIO
import numpy as np
import plotly.graph_objects as go
from collections import Counter
import math

# Function to calculate the probability distribution for a column
def probability_distribution(column):
    total = len(column)
    return {residue: count / total for residue, count in Counter(column).items()}

# Function to calculate mutual information
def mutual_information(distribution1, distribution2, joint_distribution):
    mi = 0.0
    for key1, prob1 in distribution1.items():
        for key2, prob2 in distribution2.items():
            joint_prob = joint_distribution.get((key1, key2), 0)
            if joint_prob > 0:
                mi += joint_prob * math.log2(joint_prob / (prob1 * prob2))
    return mi

# Load aligned sequences
aligned_sequences = AlignIO.read("aligned_A.fasta", "fasta")

# Convert aligned sequences to a list of strings
seq_str = [str(record.seq) for record in aligned_sequences]

# Transpose to get columns
transposed = np.array([list(seq) for seq in seq_str]).T

# Calculate mutual information for each pair of positions
num_positions = len(transposed)
mi_matrix = np.zeros((num_positions, num_positions))

for i in range(num_positions):
    for j in range(num_positions):  # Compute for all pairs
        dist1 = probability_distribution(transposed[i])
        dist2 = probability_distribution(transposed[j])
        joint_dist = Counter(zip(transposed[i], transposed[j]))
        mi = mutual_information(dist1, dist2, joint_dist)
        mi_matrix[i, j] = mi
    if (i + 1) % 10 == 0:
        print(f"Mutual information calculations completed for {i + 1} positions.")

# Create a heatmap using Plotly
fig = go.Figure(data=go.Heatmap(
    z=mi_matrix,
    x=[f"Pos {i+1}" for i in range(num_positions)],
    y=[f"Pos {j+1}" for i in range(num_positions)],
    colorscale='Cividis'))  # Using Cividis colorscale

fig.update_layout(
    title='Mutual Information Heatmap between Positions in Protein Alignment',
    xaxis_title="Position",
    yaxis_title="Position",
    width=800,
    height=800)

fig.show()

In [ ]:
from Bio import AlignIO
from collections import Counter

# Define ANSI escape sequences for colors
RED = '\033[91m'
ENDC = '\033[0m'

# Load the alignment file
alignment = AlignIO.read("aligned_B.fasta", "fasta")

# Generate consensus sequence list
consensus_seq = []
for i in range(len(alignment[0])):
    column = [record.seq[i] for record in alignment]  # Extract the column
    most_common = Counter(column).most_common(1)[0][0]
    consensus_seq.append(most_common)

# Initialize the colored consensus sequence as an empty string
colored_consensus = ""

# Iterate over the consensus sequence and add color
for position, amino_acid in enumerate(consensus_seq, start=1):
    # Highlight every 10th amino acid
    if position % 10 == 0:
        colored_consensus += RED + amino_acid + ENDC
    else:
        colored_consensus += amino_acid

    # Every 100th character (including gaps), insert a newline
    if position % 50 == 0:
        colored_consensus += '\n'

# Print the colored consensus sequence
print(colored_consensus)

In [ ]:
from Bio import AlignIO
import numpy as np
import math
from collections import Counter

# Function definitions (shannon_entropy, normalize_entropy) remain the same as before

# Load the aligned sequences using Biopython
aligned_sequences = AlignIO.read('aligned_A.fasta', 'fasta')

# Convert aligned sequences to a list of strings
aligned_seq_str = [str(record.seq) for record in aligned_sequences]

# Transpose the alignment for column-wise processing
transposed_alignment = np.array([list(seq) for seq in aligned_seq_str]).T

# Calculate entropy for each column
entropies = [shannon_entropy(column) for column in transposed_alignment]

# Find the number of unique residues (with and without gaps)
unique_residues_with_gaps = len(set(''.join([''.join(column) for column in transposed_alignment])))
unique_residues_without_gaps = len(set(''.join([''.join(column) for column in transposed_alignment])) - set('-'))

# Normalized entropy values
normalized_entropies_with_gaps = normalize_entropy(entropies, unique_residues_with_gaps)
normalized_entropies_without_gaps = normalize_entropy(entropies, unique_residues_without_gaps)

# Function to find top 10 positions with lowest entropy excluding gaps
def find_lowest_entropy_positions(normalized_entropies, alignment, num_positions=10):
    sorted_positions = sorted(range(len(normalized_entropies)), key=lambda i: normalized_entropies[i])
    lowest_entropy_positions = []
    for pos in sorted_positions:
        if len(lowest_entropy_positions) >= num_positions:
            break
        column = set(alignment[pos])
        # Exclude positions that are only gaps
        if '-' in column:
            column.remove('-')
        if column:  # If there are amino acids in the column
            lowest_entropy_positions.append((pos, normalized_entropies[pos], column))
    return lowest_entropy_positions

# Find top 10 positions for both cases
top_positions_with_gaps = find_lowest_entropy_positions(normalized_entropies_with_gaps, transposed_alignment)
top_positions_without_gaps = find_lowest_entropy_positions(normalized_entropies_without_gaps, transposed_alignment)

# Print the positions, their normalized entropy values, and the corresponding amino acids
print("Top 10 positions with lowest entropy (including gaps):")
for pos, entropy, amino_acids in top_positions_with_gaps:
    print(f"Position {pos + 1}: Normalized Entropy = {entropy:.4f}, Amino Acids = {', '.join(amino_acids)}")

print("\nTop 10 positions with lowest entropy (excluding gaps):")
for pos, entropy, amino_acids in top_positions_without_gaps:
    print(f"Position {pos + 1}: Normalized Entropy = {entropy:.4f}, Amino Acids = {', '.join(amino_acids)}")

In [ ]:
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
from Bio import AlignIO

# Function to encode amino acids as integers
def encode_amino_acids(seq, aa_index):
    return np.array([aa_index.get(aa, -1) for aa in seq])

# Function to calculate the covariance matrix using CuPy
def calculate_covariance_matrix(alignment, aa_index):
    num_sequences, seq_length = alignment.shape
    aa_freq = cp.zeros((seq_length, len(aa_index)))  # Using CuPy

    for i in range(seq_length):
        for aa, idx in aa_index.items():
            aa_freq[i, idx] = cp.sum(alignment[:, i] == idx) / num_sequences

    covariance_matrix = cp.zeros((seq_length, seq_length))
    for i in range(seq_length):
        for j in range(seq_length):
            if i != j:
                for idx1 in aa_index.values():
                    for idx2 in aa_index.values():
                        p_ij = cp.sum((alignment[:, i] == idx1) & (alignment[:, j] == idx2)) / num_sequences
                        p_i = aa_freq[i, idx1]
                        p_j = aa_freq[j, idx2]
                        covariance_matrix[i, j] += p_ij - p_i * p_j
    return covariance_matrix

# Load the alignment using BioPython and NumPy
alignment = AlignIO.read("aligned_A.fasta", "fasta")
aa_index = {aa: idx for idx, aa in enumerate('ACDEFGHIKLMNPQRSTVWY')}  # Map amino acids to indices
encoded_alignment = np.array([encode_amino_acids(rec.seq, aa_index) for rec in alignment])

# Transfer the encoded alignment to CuPy for GPU computation
alignment_gpu = cp.array(encoded_alignment)

# Calculate the covariance matrix using CuPy
cov_matrix_gpu = calculate_covariance_matrix(alignment_gpu, aa_index)

# Convert the covariance matrix back to NumPy for plotting
cov_matrix_np = cp.asnumpy(cov_matrix_gpu)

# Plotting the heatmap
plt.figure(figsize=(10, 10))
plt.imshow(cov_matrix_np, cmap='hot', interpolation='nearest')
plt.title("Covariance Matrix Heatmap")
plt.xlabel("Residue Position")
plt.ylabel("Residue Position")
plt.colorbar()
plt.show()

## HLA-B 

In [ ]:
from Bio import AlignIO
import numpy as np
import math
from collections import Counter

# Function to calculate Shannon entropy for a given column in the alignment
def shannon_entropy(column):
    counts = Counter(column)
    probabilities = [freq / len(column) for freq in counts.values()]
    entropy = -sum(p * math.log2(p) if p > 0 else 0 for p in probabilities)
    return entropy

# Function to normalize entropy based on the number of residues (with and without gaps)
def normalize_entropy(entropies, num_residues):
    max_entropy = math.log2(num_residues)
    normalized_entropies = [e / max_entropy for e in entropies]
    return normalized_entropies

# Load the aligned sequences using Biopython
aligned_sequences = AlignIO.read('aligned_B.fasta', 'fasta')

# Convert aligned sequences to a list of strings for easier processing
aligned_seq_str = [str(record.seq) for record in aligned_sequences]

# Transpose the alignment for column-wise processing
transposed_alignment = np.array([list(seq) for seq in aligned_seq_str]).T

# Calculate entropy for each column
entropies = [shannon_entropy(column) for column in transposed_alignment]

# Find the number of unique residues (with and without gaps)
unique_residues_with_gaps = len(set(''.join([''.join(column) for column in transposed_alignment])))
unique_residues_without_gaps = len(set(''.join([''.join(column) for column in transposed_alignment])) - set('-'))

# Normalized entropy values
normalized_entropies_with_gaps = normalize_entropy(entropies, unique_residues_with_gaps)
normalized_entropies_without_gaps = normalize_entropy(entropies, unique_residues_without_gaps)

# Print entropy values for each position
for i, (entropy, norm_with_gaps, norm_without_gaps) in enumerate(zip(entropies, normalized_entropies_with_gaps, normalized_entropies_without_gaps)):
    print(f"Position {i+1}: Entropy = {entropy:.4f}, Normalized (with gaps) = {norm_with_gaps:.4f}, Normalized (without gaps) = {norm_without_gaps:.4f}")

# Overall normalized entropy
overall_entropy_with_gaps = np.mean(normalized_entropies_with_gaps)
overall_entropy_without_gaps = np.mean(normalized_entropies_without_gaps)

print(f"\nOverall Normalized Entropy (with gaps): {overall_entropy_with_gaps:.4f}")
print(f"Overall Normalized Entropy (without gaps): {overall_entropy_without_gaps:.4f}")

In [ ]:
from Bio import AlignIO
from collections import Counter

# Define ANSI escape sequences for colors
RED = '\033[91m'
ENDC = '\033[0m'

# Load the alignment file
alignment = AlignIO.read("aligned_C.fasta", "fasta")

# Generate consensus sequence list
consensus_seq = []
for i in range(len(alignment[0])):
    column = [record.seq[i] for record in alignment]  # Extract the column
    most_common = Counter(column).most_common(1)[0][0]
    consensus_seq.append(most_common)

# Initialize the colored consensus sequence as an empty string
colored_consensus = ""

# Iterate over the consensus sequence and add color
for position, amino_acid in enumerate(consensus_seq, start=1):
    # Highlight every 10th amino acid
    if position % 10 == 0:
        colored_consensus += RED + amino_acid + ENDC
    else:
        colored_consensus += amino_acid

    # Every 100th character (including gaps), insert a newline
    if position % 50 == 0:
        colored_consensus += '\n'

# Print the colored consensus sequence
print(colored_consensus)

In [ ]:
from Bio import AlignIO
import numpy as np
import math
from collections import Counter

# Function definitions (shannon_entropy, normalize_entropy) remain the same as before

# Load the aligned sequences using Biopython
aligned_sequences = AlignIO.read('aligned_B.fasta', 'fasta')

# Convert aligned sequences to a list of strings
aligned_seq_str = [str(record.seq) for record in aligned_sequences]

# Transpose the alignment for column-wise processing
transposed_alignment = np.array([list(seq) for seq in aligned_seq_str]).T

# Calculate entropy for each column
entropies = [shannon_entropy(column) for column in transposed_alignment]

# Find the number of unique residues (with and without gaps)
unique_residues_with_gaps = len(set(''.join([''.join(column) for column in transposed_alignment])))
unique_residues_without_gaps = len(set(''.join([''.join(column) for column in transposed_alignment])) - set('-'))

# Normalized entropy values
normalized_entropies_with_gaps = normalize_entropy(entropies, unique_residues_with_gaps)
normalized_entropies_without_gaps = normalize_entropy(entropies, unique_residues_without_gaps)

# Function to find top 10 positions with lowest entropy excluding gaps
def find_lowest_entropy_positions(normalized_entropies, alignment, num_positions=10):
    sorted_positions = sorted(range(len(normalized_entropies)), key=lambda i: normalized_entropies[i])
    lowest_entropy_positions = []
    for pos in sorted_positions:
        if len(lowest_entropy_positions) >= num_positions:
            break
        column = set(alignment[pos])
        # Exclude positions that are only gaps
        if '-' in column:
            column.remove('-')
        if column:  # If there are amino acids in the column
            lowest_entropy_positions.append((pos, normalized_entropies[pos], column))
    return lowest_entropy_positions

# Find top 10 positions for both cases
top_positions_with_gaps = find_lowest_entropy_positions(normalized_entropies_with_gaps, transposed_alignment)
top_positions_without_gaps = find_lowest_entropy_positions(normalized_entropies_without_gaps, transposed_alignment)

# Print the positions, their normalized entropy values, and the corresponding amino acids
print("Top 10 positions with lowest entropy (including gaps):")
for pos, entropy, amino_acids in top_positions_with_gaps:
    print(f"Position {pos + 1}: Normalized Entropy = {entropy:.4f}, Amino Acids = {', '.join(amino_acids)}")

print("\nTop 10 positions with lowest entropy (excluding gaps):")
for pos, entropy, amino_acids in top_positions_without_gaps:
    print(f"Position {pos + 1}: Normalized Entropy = {entropy:.4f}, Amino Acids = {', '.join(amino_acids)}")

In [ ]:
from Bio import AlignIO
import numpy as np
import math
from collections import Counter

# Function to calculate probability distribution for a given column
def probability_distribution(column):
    counts = Counter(column)
    total = sum(counts.values())
    return {residue: count / total for residue, count in counts.items()}

# Function to calculate cross-entropy between two columns
def cross_entropy(dist1, dist2):
    residues = set(dist1.keys()).union(dist2.keys())
    return -sum(dist1.get(residue, 0) * math.log2(dist2.get(residue, 0) if dist2.get(residue, 0) > 0 else 1) for residue in residues)

# Load the aligned sequences
try:
    aligned_sequences = AlignIO.read('aligned_B.fasta', 'fasta')
except Exception as e:
    print(f"Error loading alignment file: {e}")
    raise

# Convert aligned sequences to a list of strings
aligned_seq_str = [str(record.seq) for record in aligned_sequences]

# Check if the alignment is correctly loaded
if not aligned_seq_str:
    print("No sequences found in the alignment.")
else:
    print(f"Loaded {len(aligned_seq_str)} sequences.")

# Transpose the alignment for column-wise processing
transposed_alignment = np.array([list(seq) for seq in aligned_seq_str]).T

# Calculate probability distributions for each column
prob_distributions = [probability_distribution(column) for column in transposed_alignment]

# Calculate cross-entropy for each pair of columns
cross_entropy_values = np.zeros((len(prob_distributions), len(prob_distributions)))
for i, dist1 in enumerate(prob_distributions):
    for j, dist2 in enumerate(prob_distributions):
        cross_entropy_values[i, j] = cross_entropy(dist1, dist2)

# Normalize cross-entropy values
max_entropy = math.log2(len(prob_distributions))
normalized_cross_entropy = cross_entropy_values / max_entropy

# Print cross-entropy values
print("Cross-Entropy Values:")
for i in range(len(cross_entropy_values)):
    for j in range(len(cross_entropy_values)):
        print(f"Cross-Entropy between positions {i+1} and {j+1}: {cross_entropy_values[i, j]:.4f}, Normalized: {normalized_cross_entropy[i, j]:.4f}")

# Create a heatmap using Plotly
fig = go.Figure(data=go.Heatmap(
    z=cross_entropy_values,
    x=[f"Pos {i+1}" for i in range(len(cross_entropy_values))],
    y=[f"Pos {j+1}" for j in range(len(cross_entropy_values))],
    colorscale='Viridis'))

fig.update_layout(
    title='Cross-Entropy Heatmap between Positions in Protein Alignment',
    xaxis_nticks=36,
    yaxis_nticks=36,    
    width=1000,  
    height=1000,  
    margin=dict(l=100, r=100, t=100, b=100)  
)

fig.show()

In [ ]:
import numpy as np
from Bio import AlignIO
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
from multiprocessing import Pool, cpu_count

def encode_sequences(sequences, alphabet='ACDEFGHIKLMNPQRSTVWY-'):
    char_to_int = {c: i for i, c in enumerate(alphabet)}
    encoded_seqs = [[char_to_int.get(char, -1) for char in seq] for seq in sequences]
    return np.array(encoded_seqs, dtype=int)

def calculate_mi_for_position(encoded_seqs, i):
    X = np.delete(encoded_seqs, i, axis=1)
    y = encoded_seqs[:, i]
    mi_score = mutual_info_classif(X, y, discrete_features=True)
    return np.insert(mi_score, i, 0)  # Insert 0 at the ith position for the diagonal

def main():
    aligned_sequences = AlignIO.read("aligned_B.fasta", "fasta")
    sequences = [str(record.seq) for record in aligned_sequences]
    encoded_seqs = encode_sequences(sequences)

    num_features = encoded_seqs.shape[1]
    mi_matrix = np.zeros((num_features, num_features))

    with Pool(processes=cpu_count()) as pool:
        results = pool.starmap(calculate_mi_for_position, [(encoded_seqs, i) for i in range(num_features)])

    for i, mi_scores in enumerate(results):
        mi_matrix[i, :] = mi_scores

    # Normalize the mutual information matrix using MinMaxScaler
    scaler = MinMaxScaler()
    mi_matrix_normalized = scaler.fit_transform(mi_matrix)

    # Create a heatmap using Plotly
    fig = go.Figure(data=go.Heatmap(
        z=mi_matrix_normalized,
        x=[f"Pos {i+1}" for i in range(num_features)],
        y=[f"Pos {i+1}" for i in range(num_features)],
        colorscale='Portland',  # Changed to Viridis for better color contrast
        zmin=0, zmax=1  # Set the scale from 0 to 1
    ))

    fig.update_layout(
        title='MinMax Scaled Mutual Information Heatmap',
        xaxis_title="Position",
        yaxis_title="Position",
        width=900,
        height=900
    )

    fig.show()

if __name__ == '__main__':
    main()


## HLA-C 

In [ ]:
from Bio import AlignIO
import numpy as np
import math
from collections import Counter

# Function to calculate Shannon entropy for a given column in the alignment
def shannon_entropy(column):
    counts = Counter(column)
    probabilities = [freq / len(column) for freq in counts.values()]
    entropy = -sum(p * math.log2(p) if p > 0 else 0 for p in probabilities)
    return entropy

# Function to normalize entropy based on the number of residues (with and without gaps)
def normalize_entropy(entropies, num_residues):
    max_entropy = math.log2(num_residues)
    normalized_entropies = [e / max_entropy for e in entropies]
    return normalized_entropies

# Load the aligned sequences using Biopython
aligned_sequences = AlignIO.read('aligned_C.fasta', 'fasta')

# Convert aligned sequences to a list of strings for easier processing
aligned_seq_str = [str(record.seq) for record in aligned_sequences]

# Transpose the alignment for column-wise processing
transposed_alignment = np.array([list(seq) for seq in aligned_seq_str]).T

# Calculate entropy for each column
entropies = [shannon_entropy(column) for column in transposed_alignment]

# Find the number of unique residues (with and without gaps)
unique_residues_with_gaps = len(set(''.join([''.join(column) for column in transposed_alignment])))
unique_residues_without_gaps = len(set(''.join([''.join(column) for column in transposed_alignment])) - set('-'))

# Normalized entropy values
normalized_entropies_with_gaps = normalize_entropy(entropies, unique_residues_with_gaps)
normalized_entropies_without_gaps = normalize_entropy(entropies, unique_residues_without_gaps)

# Print entropy values for each position
for i, (entropy, norm_with_gaps, norm_without_gaps) in enumerate(zip(entropies, normalized_entropies_with_gaps, normalized_entropies_without_gaps)):
    print(f"Position {i+1}: Entropy = {entropy:.4f}, Normalized (with gaps) = {norm_with_gaps:.4f}, Normalized (without gaps) = {norm_without_gaps:.4f}")

# Overall normalized entropy
overall_entropy_with_gaps = np.mean(normalized_entropies_with_gaps)
overall_entropy_without_gaps = np.mean(normalized_entropies_without_gaps)

print(f"\nOverall Normalized Entropy (with gaps): {overall_entropy_with_gaps:.4f}")
print(f"Overall Normalized Entropy (without gaps): {overall_entropy_without_gaps:.4f}")

In [ ]:
from Bio import AlignIO
from collections import Counter

def format_output(consensus, line_length=60):
    """Format the positions and consensus sequence into two aligned lines."""
    formatted_str = ""
    for i in range(0, len(consensus), line_length):
        # Generate the position line
        pos_line = ''.join(str(j) if j % 10 == 0 else ' ' for j in range(i, i + line_length))

        # Generate the consensus sequence line
        seq_line = ''.join(consensus[i:i+line_length])

        # Add the two lines to the formatted string
        formatted_str += pos_line + '\n' + seq_line + '\n\n'
    return formatted_str

# Load the alignment file
alignment = AlignIO.read("aligned_C.fasta", "fasta")

# Initialize the consensus sequence list
consensus_seq = []

# Iterate over each column in the alignment
for i in range(len(alignment[0])):
    column = [record.seq[i] for record in alignment]  # Extract the column
    most_common = Counter(column).most_common(1)[0][0]  # Find the most common character in the column
    consensus_seq.append(most_common)

# Format the sequence and positions for better readability
formatted_output = format_output(consensus_seq)

# Print the formatted consensus sequence with positions
print(formatted_output)

In [ ]:
from Bio import AlignIO
import numpy as np
import math
from collections import Counter

# Function definitions (shannon_entropy, normalize_entropy) remain the same as before

# Load the aligned sequences using Biopython
aligned_sequences = AlignIO.read('aligned_C.fasta', 'fasta')

# Convert aligned sequences to a list of strings
aligned_seq_str = [str(record.seq) for record in aligned_sequences]

# Transpose the alignment for column-wise processing
transposed_alignment = np.array([list(seq) for seq in aligned_seq_str]).T

# Calculate entropy for each column
entropies = [shannon_entropy(column) for column in transposed_alignment]

# Find the number of unique residues (with and without gaps)
unique_residues_with_gaps = len(set(''.join([''.join(column) for column in transposed_alignment])))
unique_residues_without_gaps = len(set(''.join([''.join(column) for column in transposed_alignment])) - set('-'))

# Normalized entropy values
normalized_entropies_with_gaps = normalize_entropy(entropies, unique_residues_with_gaps)
normalized_entropies_without_gaps = normalize_entropy(entropies, unique_residues_without_gaps)

# Function to find top 10 positions with lowest entropy excluding gaps
def find_lowest_entropy_positions(normalized_entropies, alignment, num_positions=10):
    sorted_positions = sorted(range(len(normalized_entropies)), key=lambda i: normalized_entropies[i])
    lowest_entropy_positions = []
    for pos in sorted_positions:
        if len(lowest_entropy_positions) >= num_positions:
            break
        column = set(alignment[pos])
        # Exclude positions that are only gaps
        if '-' in column:
            column.remove('-')
        if column:  # If there are amino acids in the column
            lowest_entropy_positions.append((pos, normalized_entropies[pos], column))
    return lowest_entropy_positions

# Find top 10 positions for both cases
top_positions_with_gaps = find_lowest_entropy_positions(normalized_entropies_with_gaps, transposed_alignment)
top_positions_without_gaps = find_lowest_entropy_positions(normalized_entropies_without_gaps, transposed_alignment)

# Print the positions, their normalized entropy values, and the corresponding amino acids
print("Top 10 positions with lowest entropy (including gaps):")
for pos, entropy, amino_acids in top_positions_with_gaps:
    print(f"Position {pos + 1}: Normalized Entropy = {entropy:.4f}, Amino Acids = {', '.join(amino_acids)}")

print("\nTop 10 positions with lowest entropy (excluding gaps):")
for pos, entropy, amino_acids in top_positions_without_gaps:
    print(f"Position {pos + 1}: Normalized Entropy = {entropy:.4f}, Amino Acids = {', '.join(amino_acids)}")

In [ ]:
from Bio import AlignIO
import numpy as np
import math
from collections import Counter

# Function to calculate probability distribution for a given column
def probability_distribution(column):
    counts = Counter(column)
    total = sum(counts.values())
    return {residue: count / total for residue, count in counts.items()}

# Function to calculate cross-entropy between two columns
def cross_entropy(dist1, dist2):
    residues = set(dist1.keys()).union(dist2.keys())
    return -sum(dist1.get(residue, 0) * math.log2(dist2.get(residue, 0) if dist2.get(residue, 0) > 0 else 1) for residue in residues)

# Load the aligned sequences
try:
    aligned_sequences = AlignIO.read('aligned_C.fasta', 'fasta')
except Exception as e:
    print(f"Error loading alignment file: {e}")
    raise

# Convert aligned sequences to a list of strings
aligned_seq_str = [str(record.seq) for record in aligned_sequences]

# Check if the alignment is correctly loaded
if not aligned_seq_str:
    print("No sequences found in the alignment.")
else:
    print(f"Loaded {len(aligned_seq_str)} sequences.")

# Transpose the alignment for column-wise processing
transposed_alignment = np.array([list(seq) for seq in aligned_seq_str]).T

# Calculate probability distributions for each column
prob_distributions = [probability_distribution(column) for column in transposed_alignment]

# Calculate cross-entropy for each pair of columns
cross_entropy_values = np.zeros((len(prob_distributions), len(prob_distributions)))
for i, dist1 in enumerate(prob_distributions):
    for j, dist2 in enumerate(prob_distributions):
        cross_entropy_values[i, j] = cross_entropy(dist1, dist2)

# Normalize cross-entropy values
max_entropy = math.log2(len(prob_distributions))
normalized_cross_entropy = cross_entropy_values / max_entropy

# Print cross-entropy values
#print("Cross-Entropy Values:")
#for i in range(len(cross_entropy_values)):
#    for j in range(len(cross_entropy_values)):
#        print(f"Cross-Entropy between positions {i+1} and {j+1}: {cross_entropy_values[i, j]:.4f}, Normalized: {normalized_cross_entropy[i, j]:.4f}")

# Create a heatmap using Plotly
fig = go.Figure(data=go.Heatmap(
    z=cross_entropy_values,
    x=[f"Pos {i+1}" for i in range(len(cross_entropy_values))],
    y=[f"Pos {j+1}" for j in range(len(cross_entropy_values))],
    colorscale='Viridis'))

fig.update_layout(
    title='Cross-Entropy Heatmap between Positions in Protein Alignment',
    xaxis_nticks=36,
    yaxis_nticks=36,    
    width=1000,  
    height=1000,  
    margin=dict(l=100, r=100, t=100, b=100)  
)

fig.show()

In [ ]:
import numpy as np
from Bio import AlignIO
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
from multiprocessing import Pool, cpu_count

def encode_sequences(sequences, alphabet='ACDEFGHIKLMNPQRSTVWY-'):
    char_to_int = {c: i for i, c in enumerate(alphabet)}
    encoded_seqs = [[char_to_int.get(char, -1) for char in seq] for seq in sequences]
    return np.array(encoded_seqs, dtype=int)

def calculate_mi_for_position(encoded_seqs, i):
    X = np.delete(encoded_seqs, i, axis=1)
    y = encoded_seqs[:, i]
    mi_score = mutual_info_classif(X, y, discrete_features=True)
    return np.insert(mi_score, i, 0)  # Insert 0 at the ith position for the diagonal

def main():
    aligned_sequences = AlignIO.read("aligned_C.fasta", "fasta")
    sequences = [str(record.seq) for record in aligned_sequences]
    encoded_seqs = encode_sequences(sequences)

    num_features = encoded_seqs.shape[1]
    mi_matrix = np.zeros((num_features, num_features))

    with Pool(processes=cpu_count()) as pool:
        results = pool.starmap(calculate_mi_for_position, [(encoded_seqs, i) for i in range(num_features)])

    for i, mi_scores in enumerate(results):
        mi_matrix[i, :] = mi_scores

    # Normalize the mutual information matrix using MinMaxScaler
    scaler = MinMaxScaler()
    mi_matrix_normalized = scaler.fit_transform(mi_matrix)

    # Create a heatmap using Plotly
    fig = go.Figure(data=go.Heatmap(
        z=mi_matrix_normalized,
        x=[f"Pos {i+1}" for i in range(num_features)],
        y=[f"Pos {i+1}" for i in range(num_features)],
        colorscale='Portland',  # Changed to Viridis for better color contrast
        zmin=0, zmax=1  # Set the scale from 0 to 1
    ))

    fig.update_layout(
        title='MinMax Scaled Mutual Information Heatmap',
        xaxis_title="Position",
        yaxis_title="Position",
        width=900,
        height=900
    )

    fig.show()

if __name__ == '__main__':
    main()